# Code to Adjust Offensive, Defensive and Net Ratings for Strength of Schedule
## Trying more stuff

In [ ]:
from sklearn.linear_model import RidgeCV
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

# from sklearn.pipeline import make_pipeline
# model = make_pipeline(StandardScaler(with_mean=False), _RidgeGCV())

teams_dict, teams_list = get_teams(league="NBA")
box_DIR = "../fdata/boxscores_team/"

In [ ]:
def time_decay(X):
    X = X.to_numpy()
    y = np.zeros(np.size(X))
    for i,x in enumerate(X):
        yj=0
        for j in range(i):
            xj = x-X[j]
            yj += np.exp(-(x-X[j]))
        y[i] = yj
    return y

In [ ]:
def get_ratings(season=2023,rest = 1):
    df1 = pd.read_csv(box_DIR + f"NBA_BoxScores_Adv_{season}.csv")
    df1 = df1.rename(
        columns={
            "offensiveRating": "ORtg",
            "defensiveRating": "DRtg",
            "netRating": "NRtg",
            "possessions": "poss",
        }
    )
    cols = ["gameId", "teamId", "ORtg", "DRtg", "NRtg", "poss"]
    df1 = df1[cols]
    df1["Win"] = df1["NRtg"] > 0
    df1["Loss"] = df1["NRtg"] < 0
    df2 = pd.read_csv(box_DIR + f"NBA_BoxScores_Standard_{season}.csv")
    df2 = df2.rename(
        columns={
            "GAME_ID": "gameId",
            "TEAM_ID": "teamId",
            "TEAM_ABBREVIATION": "teamTricode",
            "GAME_DATE": "gameDate",
            "TEAM_NAME": "teamName",
            "MATCHUP": "Matchup",
        }
    )
    cols2 = ["gameId", "teamId", "teamTricode", "gameDate","Matchup"]
    df2 = df2[cols2]
    df2["Home"] = np.where(~df2["Matchup"].str.contains("@"), 1, 0)
    df2["Away"] = np.where(df2["Matchup"].str.contains("@"), 1, 0)
    df2["gameId"] = df2["gameId"].astype(int)
    df3 = pd.merge(df2, df1, on=["gameId", "teamId"])
    df3 = df3.rename(columns={"gameDate": "Date"})
    df3["Date"] = pd.to_datetime(df3["Date"], format="%Y-%m-%d")
    df4 = df3.sort_values(by=["teamTricode","Date"])
    df4["Date_D"] = (df4["Date"] - df4["Date"].iloc[0]).dt.days
    teams = df4["teamTricode"].unique()
    dfa = []
    for team in teams:
        df5 = df4[df4["teamTricode"] == team].reset_index(drop=True)
        df5["Rest"] = time_decay(df5['Date_D'])
        df5["Rest"] = df5["Rest"].round(5)
        dfa.append(df5)
    df5 = pd.concat(dfa)
    df5 = df5.reset_index(drop=True)
    cols = [
        "gameId",
        "teamTricode",
        "teamId",
        "ORtg",
        "DRtg",
        "NRtg",
        "poss",
        "Home",
        "Away",
        "Rest",
    ]
    df6 = df5[cols]
    df6.iloc[:, 1:] = df6.iloc[:, 1:].astype(str)
    df7 = df6.groupby("gameId")[cols[1:]].agg(", ".join).reset_index()
    df8 = df7.copy()
    df8[["team1", "team2"]] = df7["teamTricode"].str.split(",", expand=True)
    df8[["tId1", "tId2"]] =   df7["teamId"].str.split(",", expand=True)
    df8[["ORtg1", "ORtg2"]] = df7["ORtg"].str.split(",", expand=True)
    df8[["DRtg1", "DRtg2"]] = df7["DRtg"].str.split(",", expand=True)
    df8[["NRtg1", "NRtg2"]] = df7["NRtg"].str.split(",", expand=True)
    df8[["poss1", "poss2"]] = df7["poss"].str.split(",", expand=True)
    df8[["rest1", "rest2"]] = df7["Rest"].str.split(",", expand=True)
    df8[["home1", "home2"]] = df7["Home"].str.split(",", expand=True)
    # df8[["away1", "away2"]] = df7["Away"].str.split(",", expand=True)
    df8 = df8.drop(columns=cols[1:])
    df9 = df7.copy()
    df9[["team2", "team1"]] = df7["teamTricode"].str.split(",", expand=True)
    df9[["tId2", "tId1"]] =   df7["teamId"].str.split(",", expand=True)
    df9[["ORtg2", "ORtg1"]] = df7["ORtg"].str.split(",", expand=True)
    df9[["DRtg2", "DRtg1"]] = df7["DRtg"].str.split(",", expand=True)
    df9[["NRtg2", "NRtg1"]] = df7["NRtg"].str.split(",", expand=True)
    df9[["poss2", "poss1"]] = df7["poss"].str.split(",", expand=True)
    df9[["rest2", "rest1"]] = df7["Rest"].str.split(",", expand=True)
    df9[["home2", "home1"]] = df7["Home"].str.split(",", expand=True)
    # df9[["away2", "away1"]] = df7["Away"].str.split(",", expand=True)
    df9 = df9.drop(columns=cols[1:])
    df10 = pd.concat([df8, df9]).sort_values(by="gameId").reset_index(drop=True)
    cols = df10.columns
    df10[cols[3:5]]   = df10[cols[3:5]].astype(int)
    df10[cols[5:15]]  = df10[cols[5:15]].astype(float)
    df10[cols[15:17]] = df10[cols[15:17]].astype(int)
    df10[cols[15:17]] = df10[cols[15:17]].astype(bool)
    data = df10.copy()
    # Old Version of Prior with using Home/Away Off rating + rest
    # and Home/Away Def rating + rest : wasn't working well
    # df1 = data.query("home1").reset_index(drop=True)
    # df1["pts1"] = df1["ORtg1"] * df1["poss1"] 
    # df1["pts2"] = df1["DRtg1"] * df1["poss1"]
    # off_prior = df1.groupby(["tId1"])[["poss1", "pts1"]].agg("sum").reset_index()
    # def_prior = df1.groupby(["tId1"])[["poss1", "pts2"]].agg("sum").reset_index()
    # off_prior["OFF"] = off_prior["pts1"] / off_prior["poss1"]
    # off_prior["OFF"] = off_prior["OFF"].round(3)
    # off_prior = off_prior[["tId1", "OFF"]]
    # def_prior["DEF"] = def_prior["pts2"] / def_prior["poss1"]
    # def_prior["DEF"] = def_prior["DEF"].round(3)
    # def_prior = def_prior[["tId1", "DEF"]]
    # def_prior = def_prior.rename(columns={"tId1":"tId2"})
    # df1 = pd.merge(df1,off_prior, on="tId1")
    # df1 = pd.merge(df1,def_prior, on="tId2")
    # df2 = data.query("home2").reset_index(drop=True)
    # df2["pts1"] = df2["ORtg1"] * df2["poss1"] 
    # df2["pts2"] = df2["DRtg1"] * df2["poss1"]
    # off_prior = df2.groupby(["tId1"])[["poss1", "pts1"]].agg("sum").reset_index()
    # def_prior = df2.groupby(["tId1"])[["poss1", "pts2"]].agg("sum").reset_index()
    # off_prior["OFF"] = off_prior["pts1"] / off_prior["poss1"]
    # off_prior["OFF"] = off_prior["OFF"].round(3)
    # off_prior = off_prior[["tId1", "OFF"]]
    # def_prior["DEF"] = def_prior["pts2"] / def_prior["poss1"]
    # def_prior["DEF"] = def_prior["DEF"].round(3)
    # def_prior = def_prior[["tId1", "DEF"]]
    # def_prior = def_prior.rename(columns={"tId1":"tId2"})
    # df2 = pd.merge(df2,off_prior, on="tId1")
    # df2 = pd.merge(df2,def_prior, on="tId2")
    # data= pd.concat([df1,df2])
    # data = data.drop(columns = ["pts1","pts2"])
    # data["OFF_P"] = (data["OFF"] - 1*data["rest1"]).round(3)
    # data["DEF_P"] = (data["DEF"] + 1*data["rest2"]).round(3)
    # # data["Prior"] = (0.5*(data["OFF_P"]+data["DEF_P"])).round(3)
    # data["Prior"] = (data["OFF"]).round(3)
    # data = data.drop(columns=["OFF","DEF","OFF_P","DEF_P"])

    # Prior with using HCA of Off Rating + rest
    # and HCA of rating + rest l
    # df1 = data.copy()
    # df1["pts1"] = df1["ORtg1"] * df1["poss1"] 
    # df1["pts2"] = df1["DRtg1"] * df1["poss1"]
    # off_p = df1.groupby(["tId1"])[["poss1", "pts1"]].agg("sum").reset_index()
    # def_p = df1.groupby(["tId1"])[["poss1", "pts2"]].agg("sum").reset_index()
    # off_p["OFF"] = off_p["pts1"] / off_p["poss1"]
    # off_p["OFF"] = off_p["OFF"].round(3)
    # off_p_all = off_p[["tId1", "OFF"]]
    # def_p["DEF"] = def_p["pts2"] / def_p["poss1"]
    # def_p["DEF"] = def_p["DEF"].round(3)
    # def_p_all = def_p[["tId1", "DEF"]]
    # df1 = data.query("home1").reset_index(drop=True)
    # df1["pts1"] = df1["ORtg1"] * df1["poss1"] 
    # df1["pts2"] = df1["DRtg1"] * df1["poss1"]
    # off_p = df1.groupby(["tId1"])[["poss1", "pts1"]].agg("sum").reset_index()
    # def_p = df1.groupby(["tId1"])[["poss1", "pts2"]].agg("sum").reset_index()
    # off_p["OFF"] = off_p["pts1"] / off_p["poss1"]
    # off_p["OFF"] = off_p["OFF"].round(3)
    # off_p_home = off_p[["tId1", "OFF"]]
    # def_p["DEF"] = def_p["pts2"] / def_p["poss1"]
    # def_p["DEF"] = def_p["DEF"].round(3)
    # def_p_home = def_p[["tId1", "DEF"]]
    # off_adv= pd.merge(off_p_home,off_p_all,on="tId1")
    # def_adv= pd.merge(def_p_home,def_p_all,on="tId1")
    # off_adv["OHCA"] = off_adv["OFF_x"] - off_adv["OFF_y"]
    # def_adv["DHCA"] = def_adv["DEF_x"] - def_adv["DEF_y"]
    # def_adv = def_adv.rename(columns={"tId1":"tId2"})
    # off_adv = off_adv.drop(columns=["OFF_x","OFF_y"])
    # def_adv = def_adv.drop(columns=["DEF_x","DEF_y"])
    # data = pd.merge(data,off_adv,on="tId1")
    # data = pd.merge(data,def_adv,on="tId2")

    # data["Prior"] = (- 10* data["rest1"]  
    #                  + 10* data["rest2"] 
    #                  + np.where(data["home1"],1,-1)*data["OHCA"]
    #                  + np.where(data["home2"],1,-1)*data["DHCA"]
    #             ).round(3)
    
    # Prior with using league avg HCA + rest
    df1 = data.copy()
    df1["pts1"] = df1["ORtg1"] * df1["poss1"] 
    df1["pts2"] = df1["DRtg1"] * df1["poss1"]
    off_all = (df1["pts1"].sum() / df1["poss1"].sum()).round(3)
    def_all = (df1["pts2"].sum() / df1["poss1"].sum()).round(3)
    df1 = data.query("home1").reset_index(drop=True)
    df1["pts1"] = df1["ORtg1"] * df1["poss1"] 
    df1["pts2"] = df1["DRtg1"] * df1["poss1"]
    off_home = (df1["pts1"].sum() / df1["poss1"].sum()).round(3)
    def_home = (df1["pts2"].sum() / df1["poss1"].sum()).round(3)
    off_adv = off_home - off_all
    def_adv = def_home - def_all
    data["Prior"] = (- rest* data["rest1"]  
                     + rest* data["rest2"] 
                     + np.where(data["home1"],1,-1)*off_adv
                     + np.where(data["home2"],1,-1)*def_adv
                ).round(3)

    return data

In [ ]:
def process_results(data, results_adj):
    data["pts1"] = data["ORtg1"] * data["poss1"]
    data["pts2"] = data["DRtg1"] * data["poss1"]
    off_prior = data.groupby(["tId1"])[["poss1", "pts1"]].agg("sum").reset_index()
    def_prior = data.groupby(["tId1"])[["poss1", "pts2"]].agg("sum").reset_index()
    off_prior["OFF"] = off_prior["pts1"] / off_prior["poss1"]
    off_prior = off_prior[["tId1", "OFF"]]
    def_prior["DEF"] = def_prior["pts2"] / def_prior["poss1"]
    def_prior = def_prior[["tId1", "DEF"]]
    results_net = pd.merge(off_prior, def_prior, on=["tId1"])
    results_net["NET"] = results_net["OFF"] - results_net["DEF"]
    results_net.rename(columns={"tId1": "tId"}, inplace=True)
    results_net = results_net.astype(float).round(2)
    results_net["tId"] = results_net["tId"].astype(int)
    ortg_mean = data["pts1"].sum() / data["poss1"].sum()
    drtg_mean = data["pts2"].sum() / data["poss1"].sum()
    results_adj["tId"] = results_adj["tId"].astype(int)
    results_comb = pd.merge(results_net, results_adj, on=["tId"])
    results_comb["aOFF"] = results_comb["aOFF"] #+ results_comb["OFF"]
    results_comb["aDEF"] = results_comb["aDEF"] #+ results_comb["DEF"]
    results_comb["aNET"] = results_comb["aNET"] #+ results_comb["NET"]
    results_comb["oSOS"] = results_comb["aOFF"] - results_comb["OFF"]
    results_comb["dSOS"] = results_comb["DEF"] - results_comb["aDEF"]
    results_comb["SOS"] = results_comb["oSOS"] + results_comb["dSOS"]
    results_comb.iloc[:, 1:] = results_comb.iloc[:, 1:].round(1)
    results = results_comb[
        ["Team", "OFF", "oSOS", "aOFF", "DEF", "dSOS", "aDEF", "NET", "SOS", "aNET"]
    ]
    # results = results_comb[["Team","OFF","DEF","NET","aOFF","aDEF","aNET"]]
    results = results.sort_values(by="aNET", ascending=0).reset_index(drop=True)
    return results, ortg_mean, drtg_mean

In [ ]:
def map_teams(row_in, teams, scale):
    t1 = row_in[0]
    t2 = row_in[1]

    rowOut = np.zeros([len(teams) * 2])
    rowOut[teams.index(t1)] = scale
    rowOut[teams.index(t2) + len(teams)] = scale

    return rowOut


def convert_to_matricies(possessions, name, teams, prior, scale=1):
    # extract only the columns we need
    # Convert the columns of player ids into a numpy matrix
    stints_x_base = possessions[["tId1", "tId2"]].to_numpy()
    # Apply our mapping function to the numpy matrix
    stint_X_rows = np.apply_along_axis(map_teams, 1, stints_x_base, teams, scale=scale)
    # Convert the column of target values into a numpy matrix
    # Convert the column of target values into a numpy matrix
    stint_Y_rows_before = possessions[name].to_numpy()
    stint_Y_rows = stint_Y_rows_before - prior

    # return matricies and possessions series
    return stint_X_rows, stint_Y_rows


# Convert lambda value to alpha needed for ridge CV


def lambda_to_alpha(lambda_value, samples):
    return (lambda_value * samples) / 2.0


# Convert RidgeCV alpha back into a lambda value


def alpha_to_lambda(alpha_value, samples):
    return (alpha_value * 2.0) / samples


def calculate_netrtg(train_x, train_y, lambdas, teams_list, prior):
    alphas = [lambda_to_alpha(l, train_x.shape[0]) for l in lambdas]
    # create a 5 fold CV ridgeCV model. Our target data is not centered at 0, so we want to fit to an intercept.
    clf = RidgeCV(alphas=alphas, cv=5, fit_intercept=True)

    # fit our training data
    model = clf.fit(
        train_x,
        train_y,
    )

    # convert our list of players into a mx1 matrix
    team_arr = np.transpose(np.array(teams_list).reshape(1, len(teams_list)))

    # extract our coefficients into the offensive and defensive parts
    coef_ = model.coef_ 
    coef_offensive_array = coef_[0 : len(teams_list)][np.newaxis].T
    coef_defensive_array = coef_[len(teams_list) : 2 * len(teams_list)][
        np.newaxis
    ].T
    # concatenate the offensive and defensive values with the playey ids into a mx3 matrix
    team_id_with_coef = np.concatenate(
        [team_arr, coef_offensive_array, coef_defensive_array], axis=1
    )
    # build a dataframe from our matrix
    teams_coef = pd.DataFrame(team_id_with_coef)
    intercept = model.intercept_
    teams_coef.columns = ["tId", "aOFF", "aDEF"]
    teams_coef["aNET"] = teams_coef["aOFF"] - teams_coef["aDEF"]
    teams_coef["aOFF"] = teams_coef["aOFF"] + intercept
    teams_coef["aDEF"] = teams_coef["aDEF"] + intercept
    teams_coef["Team"] = teams_coef["tId"].map(teams_dict)
    results = teams_coef[["tId", "Team", "aOFF", "aDEF", "aNET"]]
    results = results.sort_values(by=["aNET"], ascending=False).reset_index(drop=True)
    return results, model, intercept

In [ ]:
# seasons = np.arange(2010,2023,1).astype(str)
# dfa = []
# for season in seasons:
#     df = get_ratings(season)
#     dfa.append(df)
# data = pd.concat(dfa)

In [ ]:
data = get_ratings(2023, rest=10)
prior = data["Prior"].to_numpy()
train_x, train_y = convert_to_matricies(data, "ORtg1", teams_list, prior, scale = 1/2)
n = 1.5
lambdas_net = [0.01 * n, 0.05 * n, 0.1 * n]
results_adj, model, intercept = calculate_netrtg(
    train_x, train_y, lambdas_net, teams_list, prior
)
results, ortg_mean, drtg_mean = process_results(data, results_adj)
results.index = results.index +1
# results.to_csv("./NBA_Adj_Ratings.csv",index=False)
print(intercept)
# results.head(5)
# results.sort_values(by="aOFF",ascending=0)
# results.sort_values(by="aDEF",ascending=1)
# results

In [ ]:
results.sort_values("SOS",ascending=True)

In [ ]:
xcvxcv

In [ ]:
var1 = "NET"
# var1 = "OFF"
# var1 = "DEF"
var2 = "a" + var1
slope, intercept, r, p, sterr = scipy.stats.linregress(x=results[var1], y=results[var2])
r2 = r**2
fig, ax = plt.subplots(1, 1)
fig = sns.regplot(
    x=var1,
    y=var2,
    data=results,
    color="black",
    scatter_kws={"color": "tab:blue"},
    ax=ax,
)
ax.text(0.05, 0.9, r"$r^2=$" + f"{round(r2,4)}", transform=ax.transAxes)
ax.set_title("Adjusted Net Ratings vs Unadjusted")
plt.savefig("../figs/team_leaders/aNET_R2_1.png")
plt.show()

In [ ]:
results["OFF_R"] = results["OFF"].rank(ascending=False).astype(int)
results["DEF_R"] = results["DEF"].rank(ascending=True).astype(int)
results["NET_R"] = results["NET"].rank(ascending=False).astype(int)
results["aOFF_R"] = results["aOFF"].rank(ascending=False).astype(int)
results["aDEF_R"] = results["aDEF"].rank(ascending=True).astype(int)
results["aNET_R"] = results["aNET"].rank(ascending=False).astype(int)

In [ ]:
df_teams = pd.read_csv("../data/NBA_teams_colors_logos.csv")
df_teams = df_teams.rename(columns={"nameTeam": "Team"})
results_plot = pd.merge(results, df_teams)

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%%R -i results_plot
results <- results_plot
library(tidyverse)
library(ggimage)
library(ggrepel)
theme_owen <- function() {
  theme_minimal(base_size = 16, base_family = "Consolas") %+replace%
    theme(
      panel.grid.minor = element_blank(),
      plot.background = element_rect(fill = "ghostwhite", color = "ghostwhite")
    )
}
p <- ggplot(
  results,
  aes(x = aOFF, y = aDEF, label = paste0("#", aNET_R, " Net"))
) +
  # geom_point(aes(size = aNRtg_Rank)) +
  scale_y_reverse() +
  geom_hline(aes(yintercept = mean(aOFF)), color = "black") +
  geom_vline(aes(xintercept = mean(aDEF)), color = "black") +
  # geom_abline(intercept = 222, slope = -1, color = "black") +
  geom_abline(slope = -1,color="black")+
  geom_image(
    aes(
      x = aOFF, y = aDEF,
      image = urlThumbnailTeam
    ),
    size = 0.1
  ) +
  # geom_text(nudge_x = 1.3, nudge_y = 0, size = 6,check_overlap = TRUE) +
  geom_text_repel(nudge_x = 1.1, nudge_y = 0.5,size=6,min.segment.length=10) +
  # geom_label(nudge_x = 1.3, nudge_y = 0, size = 6) +
  theme_owen() +
  theme(
    plot.title.position = "plot",
    plot.title = element_text(face = "bold", size = 24, hjust = 0.5),
    plot.margin = margin(10, 10, 15, 10),
    plot.subtitle = element_text(size = 18),
    plot.caption = element_text(size = 14)
  ) +
  theme(
    axis.text.x = element_text(size = 14, face = "bold", color = "black"),
    axis.text.y = element_text(size = 14, face = "bold", color = "black"),
    axis.title.x = element_text(size = 18, face = "bold", colour = "black"),
    axis.title.y = element_text(size = 18, face = "bold", colour = "black")
  ) +
  labs(
    title = paste0("Adjusted Efficiency Landscape as of ", format(Sys.Date(), format = "%B %d, %Y")),
    x = "Adjusted Offensive Rating", y = "Adjusted Defensive Rating",
    subtitle = "Net Ratings here are adjusted for Strength of Schedule",
    caption = "@SravanNBA"
  )
ggsave("../figs/team_ratings/Adjusted_TRatings.png", p, w = 10 * 1.5, h = 8 * 1.5, dpi = 300)

In [ ]:
r1 = results[
    ["Team", "aOFF_R", "aDEF_R", "aNET_R", "OFF_R", "DEF_R", "NET_R"]
].reset_index(drop=True)

In [ ]:
r1["aOFF_m"] = r1["aOFF_R"] - r1["OFF_R"]
r1["aDEF_m"] = r1["aDEF_R"] - r1["DEF_R"]
r1["aNET_m"] = r1["aNET_R"] - r1["NET_R"]

In [ ]:
r2 = r1[
    ["Team", "aOFF_R", "aOFF_m", "aDEF_R", "aDEF_m", "aNET_R", "aNET_m"]
].reset_index(drop=True)

In [ ]:
%%R -i r2
library(tidyverse)
library(gt)
df <- r2
df %>% 
  gt()%>%
  tab_header(
    title = md("**NBA Adjusted Rating Movement 2023-24**"),
    subtitle = "R: Rank, M:Movement" 
    ) %>%
    data_color(columns = c(aOFF_m,aDEF_m,aNET_m), palette = c("red", "green")) %>%
    cols_align(align = "center",columns = c(aOFF_R,aOFF_m,aDEF_R,aDEF_m,aNET_R,aNET_m))  %>%
    cols_label(
      aOFF_R = "R", aDEF_R = "R", aNET_R = "R",
      aOFF_m = "M", aDEF_m = "M", aNET_m = "M"
    ) %>%
    tab_spanner(
      label = "OFF",
      columns = c(aOFF_R, aOFF_m)
    ) %>%
    tab_spanner(
      label = "DEF",
      columns = c(aDEF_R, aDEF_m)
    ) %>%
    tab_spanner(
      label = "NET",
      columns = c(aNET_R, aNET_m)
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 20,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_source_note(
    source_note = "Movement is calculated as adjusted rank - unadjusted rank" ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/team_leaders/Teams_aNET_movement.png",zoom=5) 

In [ ]:
%%R -i results 
library(tidyverse)
library(gt)
df <- results
df %>% 
  gt()%>%
  # cols_move(OFF_R,OFF) %>% cols_move(DEF_R,DEF) %>% cols_move(NET_R,NET) %>%
  # cols_move(aOFF_R,aOFF) %>% cols_move(aDEF_R,aDEF) %>% cols_move(aNET_R,aNET) %>%
  tab_header(
    title = md("**NBA Adjusted Net Rating Leaders 2023-24**"),
    subtitle = md("The Offensive, Defensive and Net Ratings are adjusted for Strength of Schedule" )
    ) %>%
    data_color(columns = c(OFF,aOFF,NET,aNET), palette = c("red", "green")) %>%
    data_color(columns = c(DEF,aDEF), palette = c("green","red")) %>%
    cols_align(align = "center",columns = c(OFF,DEF,NET,aOFF,aDEF,aNET))  %>%
    cols_label(
      OFF_R = "#",DEF_R = "#",NET_R = "#",aOFF_R = "#",aDEF_R = "#",aNET_R = "#"
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 20,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_spanner(
      label = "Unadjusted",
      columns = c(OFF, OFF_R, DEF, DEF_R, NET, NET_R)
    ) %>%
    tab_spanner(
      label = "Adjusted",
      columns = c(aOFF, aOFF_R, aDEF, aDEF_R, aNET, aNET_R)
    ) %>%
    tab_source_note(
    source_note = "SOS adjustments done using Ridge Regression Method to estimate Off and Def Ratings" ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/team_leaders/Teams_aNET.png",zoom=5) 
